In [1]:
!pip -qq install git+https://github.com/huggingface/transformers
#!pip install -q datasets loralib sentencepiece
!pip -qq install bitsandbytes accelerate xformers einops
!pip -qq install hf_transfer

In [2]:
!nvidia-smi

Thu Feb 22 19:34:01 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   27C    P0    61W / 400W |   8789MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Gemma - 7B

In [3]:
from huggingface_hub import login

login()

In [ ]:
# from huggingface_hub import interpreter_login

# interpreter_login()

In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os


os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [5]:
torch.__version__

'2.2.0+cu121'

In [6]:
import os

os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [8]:
torch.set_default_device("cuda")

In [9]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", # 'google/gemma-2b-it
                                             quantization_config=quantization_config,
                                             low_cpu_mem_usage=True,
                                             torch_dtype="auto",
                                             device_map="auto"
                                             )

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
tokenizer.vocab_size

256000

In [11]:
chat = [
    { "role": "user", "content": "What is the difference between LLaMAs, Alpacas, and Vicunas" },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt


'<start_of_turn>user\nWhat is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>\n<start_of_turn>model\n'

In [12]:
inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")

outputs = model.generate(input_ids=inputs.to(model.device),
                         max_new_tokens=512)

In [14]:
from IPython.display import Markdown, display
display(Markdown("## Gemma - 7B 4Bit"))

text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
# text = text.replace(prompt, '', 1)
display(Markdown(text))

## Gemma - 7B 4Bit

user
What is the difference between LLaMAs, Alpacas, and Vicunas
model
Sure, here is the difference between LLamas, Alpacas, and Vicunas:

**LLamas:**

* **Origin:** North America
* **Coat:** Long, dense, soft, and warm
* **Color:** Gray, brown, black, white
* **Temperament:** Friendly, calm, and easy to train
* **Uses:** Wool production, meat production, and companionship

**Alpacas:**

* **Origin:** South America
* **Coat:** Long, dense, soft, and warm
* **Color:** White, fawn, gray, brown
* **Temperament:** Friendly, calm, and intelligent
* **Uses:** Wool production, meat production, and companionship

**Vicunas:**

* **Origin:** South America
* **Coat:** Long, dense, soft, and warm
* **Color:** White, fawn, gray, brown
* **Temperament:** Friendly, calm, and intelligent
* **Uses:** Wool production, meat production, and companionship

The main differences between LLamas, Alpacas, and Vicunas are their origin, coat color, temperament, and uses. LLamas are North American animals, while Alpacas and Vicunas are South American animals. LLamas have a wide range of coat colors, while Alpacas and Vicunas have a more limited range of coat colors. LLamas are generally friendly and easy to train, while Alpacas and Vicunas are also friendly and easy to train. LLamas are primarily used for wool production and meat production, while Alpacas and Vicunas are primarily used for wool production and meat production.

In [22]:
import textwrap

def wrap_text(text, width=90):
    lines = text.split("\n")
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = "\n".join(wrapped_lines)
    return wrapped_text

In [23]:
def generate(input_text, system_prompt="", max_length=512):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"

    messages = [
        {"role": "user", "content": f"{system_prompt} \n\n {input_text}"}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")

    outputs = model.generate(
        input_ids = inputs.to(model.device),
        max_new_tokens = max_length,
        do_sample = True,
        temperature = 0.1,
        top_k = 50,
    )

    text = tokenizer.decode(
        outputs[0], 
        skip_special_tokens=True, 
        clean_up_tokenization_spaces=True
    )
    text = text.replace(f"user\n {system_prompt} \n\n {input_text} \n model", "", 1)
    wrappped_text = wrap_text(text)
    display(Markdown(wrappped_text))




In [24]:
generate(
    "Write a detailed analogy between mathematics and a lighthouse.",
    system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
    max_length=512
)

user
You are Gemma, a large language model trained by Google. Write out your reasoning step-by-
step to be sure you get the right answers!

 Write a detailed analogy between mathematics and a lighthouse.
model
**Reasoning:**

**Step 1: Identify the key similarities between mathematics and a lighthouse.**

* **Mathematics** is a systematic body of knowledge that deals with numbers, shapes,
space, and other abstract concepts.
* **Lighthouse** is a tall structure that is used to guide ships at night.

**Step 2: Identify the key differences between mathematics and a lighthouse.**

* **Mathematics** is an abstract discipline, while a lighthouse is a physical structure.
* **Mathematics** is a theoretical field, while a lighthouse is a practical application of
mathematics.

**Step 3: Draw the analogy.**

Mathematics is like a lighthouse because it is a system of knowledge that provides
guidance and direction. Just as a lighthouse is a beacon that guides ships, mathematics is
a beacon that guides people in understanding the world.

**Conclusion:**

The analogy between mathematics and a lighthouse is appropriate because both are systems
of knowledge and guidance. Mathematics is a vast and complex field of study that provides
a framework for understanding and solving a wide range of problems. Similarly, a
lighthouse is a tall structure that provides a clear and visible guide for ships.

In [25]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)

user
You are Gemma, a large language model trained by Google. Write out your reasoning step-by-
step to be sure you get the right answers!

 Write a detailed analogy between mathematics and a music.
model
**Reasoning:**

**Step 1: Identify the similarities between mathematics and music.**

* **Structure:** Mathematics and music both have a structured framework. In mathematics,
there are rules for arithmetic, geometry, and algebra. In music, there are rules for
pitch, rhythm, and melody.
* **Elements:** Both disciplines have elements that are essential to their respective
fields. In mathematics, there are numbers, variables, and formulas. In music, there are
notes, chords, and rhythms.
* **Expression:** Mathematics and music are both expressive mediums. In mathematics,
formulas can be used to express complex ideas. In music, composers use notes and rhythms
to express emotions and stories.

**Step 2: Identify the differences between mathematics and music.**

* **Purpose:** Mathematics is primarily used for problem-solving and analysis. Music is
primarily used for entertainment and emotional expression.
* **Language:** Mathematics uses a precise language of symbols and notation. Music uses a
language of notes, chords, and rhythms.
* **Performance:** Mathematics is typically performed in a theoretical setting. Music is
performed live, in front of an audience.

**Step 3: Draw the analogy.**

Mathematics and music are two different disciplines that share some similarities and
differences

CPU times: user 17.5 s, sys: 126 ms, total: 17.7 s
Wall time: 17.7 s


In [26]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         system_prompt="Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)

user
Write out your reasoning step-by-step to be sure you get the right answers!

 What is the difference between a Llama, Vicuna and an Alpaca?
model
**Reasoning:**

**1. Identify the characteristics of each animal:**

- **Llama:**
   - Thick fur, long legs, docile temperament, and a distinctive hum.
- **Vicuna:**
   - Soft, lustrous fleece, fine wool, small stature, and a gentle temperament.
- **Alpaca:**
   - Dense fleece, soft wool, medium stature, and a calm temperament.

**2. Compare the characteristics:**

- **Llama:** Known for its thick fur and docile temperament, which are not necessarily
desirable for breeding purposes.
- **Vicuna:** Distinguished by its soft, lustrous fleece and fine wool, making it highly
sought-after for its exceptional quality.
- **Alpaca:** Distinguished by its dense fleece, soft wool, and calm temperament, making
it valuable for breeding and fiber production.

**3. Identify the key differences:**

- **Fur:** Llama has thick fur, Vicuna has soft fleece, Alpaca has dense fleece.
- **Temperament:** Llama is docile, Vicuna is gentle, Alpaca is calm.
- **Wool:** Vicuna has fine wool, Alpaca has soft wool.
- **Size:** Llama has long legs, Vicuna has small stature, Alpaca has medium stature.
- **Purpose:** Llama is not primarily bred for fiber production, Vicuna is bred for its
exceptional fleece, Alpaca is bred for fiber production and companionship.

**Conclusion:**

The primary difference between a Llama, Vicuna, and Alpaca lies in their respective
characteristics and purposes. Llama has thick fur and a docile temperament, Vicuna has
soft fleece and a gentle temperament, while Alpaca has dense fleece, soft wool, and a calm
temperament.

CPU times: user 24.8 s, sys: 153 ms, total: 25 s
Wall time: 25 s


In [27]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)

user
You are Gemma, a large language model trained by Google. Write out your reasoning step-by-
step to be sure you get the right answers!

 Write a short email to Sam Altman giving reasons to open source GPT-4
model
**Subject: Reasons for Open Sourcing GPT-4**

**Dear Sam Altman,**

I understand the importance of keeping GPT-4 confidential, given its competitive edge and
potential for commercialization. However, I believe that opening sourcing GPT-4 would
significantly benefit the research and development community, while also promoting
transparency and accountability.

**Reasons for Open Sourcing:**

**1. Accelerating Research:**
- Open sourcing GPT-4 would make it accessible to a wide range of researchers, enabling
them to experiment with the model, improve its performance, and develop new applications.
- This would accelerate the pace of research and foster innovation.

**2. Fostering Collaboration:**
- Open sourcing GPT-4 would encourage collaboration among researchers, allowing them to
share ideas, code, and data.
- This would foster a sense of community and accelerate progress.

**3. Promoting Transparency:**
- Open sourcing would make it easier for experts and the public to scrutinize GPT-4's
algorithms and evaluate its fairness and bias.
- This would enhance transparency and accountability.

**4. Accelerating Deployment:**
- Open sourcing GPT-4 would make it easier for developers to integrate the model into
their applications and services.
- This would accelerate the adoption and utilization of GPT-4.

**5. Creating a Legacy:**
- Open sourcing GPT-4 would create a lasting legacy for the research community, inspiring
future generations of engineers and scientists.
- It would also promote the development of new tools and resources for natural language
processing.

**Conclusion:**

While I understand the need for confidentiality, I believe that the benefits of open
sourcing GPT-4 outweigh the potential risks. By fostering collaboration, accelerating
research, promoting transparency, and fostering a legacy, open sourcing would advance the
field of natural language processing and benefit society as a whole.

**Sincerely,**
**Gemma**

CPU times: user 26.7 s, sys: 211 ms, total: 26.9 s
Wall time: 26.9 s


In [28]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder, please think carefully and write the following code:")

user
You are a genius python coder, please think carefully and write the following code:

 ```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """
model
```python
import math

def print_prime(n):
   """
   Print all primes between 1 and n
   """

   Primes = []
   for num in range(1, int(n**0.5) + 1):
       is_prime = True
       for p in primes:
           if num % p == 0:
               is_prime = False
       if is_prime and num not in primes:
           Primes.append(num)

   print(Primes)
```

**Explanation:**

* The function `print_prime` takes an integer `n` as input.
* It uses a list called `Primes` to store all prime numbers.
* It iterates over the range from 1 to the square root of `n`.
* For each number `num`, it checks if it is prime by checking if it is divisible by any
number in the `Primes` list.
* If `num` is prime and it is not already in the `Primes` list, it is added to the list.
* Finally, the `Primes` list is printed.

**Example Usage:**

```python
print_prime(100)
```

**Output:**

```
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 33, 37, 41, 43, 47, 51, 53, 59, 61, 63, 65, 67,
69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99]
```

**Note:**

* This algorithm is efficient for small values of `n`, but it can be slow for large
values.
* The time complexity of the algorithm is O(n) where `n` is the upper bound of the range.
* The space complexity is O(n) as well, since the algorithm uses a list of size n to store
the primes.

In [29]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         system_prompt="You are Gemma, a large language model trained by Google. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)

user
You are Gemma, a large language model trained by Google. Write out your reasoning step-by-
step to be sure you get the right answers!

 Answer the following question by reasoning step by step. The cafeteria had 23 apples. If
they used 20 for lunch, and bought 6 more, how many apple do they have?
model
**Step 1: Subtract 20 from 23.**

23 - 20 = 3

**Step 2: Add 6 to the remaining 3.**

3 + 6 = 9

Therefore, the cafeteria has a total of 9 apples left.